# Silver Table Updates

We have processed data from the Bronze table to the Silver table.
We need to do some updates to ensure high quality in the Silver
table.

😎 We're reading _from_ the Delta table now because a Delta table can be both a source AND a sink.

## Notebook Objective

In this notebook we:
1. Harden the Raw to Bronze and Bronze to Silver Steps we wrote in a
   previous notebook.
1. Diagnose data quality issues.
1. Update the broken readings in the Silver table.
1. Handle late-arriving data.

## Step Configuration

In [0]:
%run ./includes/configuration

Out[3]: DataFrame[]

No running streams.


## Import Operation Functions

In [0]:
%run ./includes/main/python/operations

### Display the Files in the Raw and Bronze Paths

In [0]:
display(dbutils.fs.ls(rawPath))

path,name,size
dbfs:/dbacademy/gaurav_chattree/dataengineering/plus/raw/health_tracker_data_2020_1.json,health_tracker_data_2020_1.json,310628
dbfs:/dbacademy/gaurav_chattree/dataengineering/plus/raw/health_tracker_data_2020_2.json,health_tracker_data_2020_2.json,284670


In [0]:
display(dbutils.fs.ls(bronzePath))

path,name,size
dbfs:/dbacademy/gaurav_chattree/dataengineering/plus/bronze/_delta_log/,_delta_log/,0
dbfs:/dbacademy/gaurav_chattree/dataengineering/plus/bronze/p_ingestdate=2022-03-25/,p_ingestdate=2022-03-25/,0


## Start Streams

Before we add new streams, let's start the streams we have previously engineered.

We will start two named streams:

- `write_raw_to_bronze`
- `write_bronze_to_silver`

### Current Delta Architecture
Next, we demonstrate everything we have built up to this point in our
Delta Architecture.

Again, we do so with composable functions included in the
file `includes/main/python/operations`.

### The Hardened Raw to Bronze Pipeline

In [0]:
rawDF = read_stream_raw(spark, rawPath)
transformedRawDF = transform_raw(rawDF)
rawToBronzeWriter = create_stream_writer(
    dataframe=transformedRawDF,
    checkpoint=bronzeCheckpoint,
    name="write_raw_to_bronze",
    partition_column="p_ingestdate",
)
rawToBronzeWriter.start(bronzePath)

Out[17]: <pyspark.sql.streaming.StreamingQuery at 0x7fd1328fe580>

### The Hardened Bronze to Silver Pipeline

In [0]:
bronzeDF = read_stream_delta(spark, bronzePath)
transformedBronzeDF = transform_bronze(bronzeDF)
bronzeToSilverWriter = create_stream_writer(
    dataframe=transformedBronzeDF,
    checkpoint=silverCheckpoint,
    name="write_bronze_to_silver",
    partition_column="p_eventdate",
)
bronzeToSilverWriter.start(silverPath)

Out[18]: <pyspark.sql.streaming.StreamingQuery at 0x7fd1328fe160>

## Show Running Streams

In [0]:
for stream in spark.streams.active:
    print(stream.name)

write_bronze_to_silver
write_raw_to_bronze


## Diagnose Data Quality Issues

It is a good idea to perform quality checking on the data - such as looking for and reconciling anomalies - as well as further transformations such as cleaning and/or enriching the data.

In a visualization in the previous notebook, we noticed:

1. the table is missing records.
1. the presence of some negative recordings even though negative heart rates are impossible.

Let's assess the extent of the negative reading anomalies.

**Exercise:** Create a Temporary View of the Broken Readings in the Silver Table

Display a count of the number of records for each day in the Silver
table where the measured heartrate is negative.

In [0]:

from pyspark.sql.functions import count
broken_readings = (
   spark.read
  .format("delta")
  .load(silverPath)
  .select(col("heartrate"), col("p_eventdate"))
  .where(col("heartrate") < 0)
  .groupby("p_eventdate")
  .agg(count("heartrate"))
  .orderBy("p_eventdate")
)

broken_readings.createOrReplaceTempView("broken_readings")


In [0]:
%sql

SELECT * FROM broken_readings

p_eventdate,count(heartrate)
2020-01-01,1
2020-01-02,1
2020-01-04,1
2020-01-06,1
2020-01-07,1
2020-01-09,2
2020-01-12,3
2020-01-13,2
2020-01-14,1
2020-01-16,2


In [0]:
%sql

SELECT SUM(`count(heartrate)`) FROM broken_readings

sum(count(heartrate))
60


We have identified two issues with the Silver table:

1. There are missing records
1. There are records with broken readings

Let's update the table.

## Update the Broken Readings
To update the broken sensor readings (heartrate less than zero), we'll interpolate using the value recorded before and after for each device. The `pyspark.sql` functions `lag()` and `lead()` will make this a trivial calculation.
In order to use these functions, we need to import the pyspark.sql.window function `Window`. This will allow us to create a date window consisting of the dates immediately before and after our missing value.

🚎Window functions operate on a group of rows, referred to as a window, and calculate a return value for each row based on the group of rows. Window functions are useful for processing tasks such as calculating a moving average, computing a cumulative statistic, or accessing the value of rows given the relative position of the current row.

We'll write these values to a temporary view called `updates`. This view will be used later to upsert values into our Silver Delta table.

[pyspark.sql window functions documentation](https://spark.apache.org/docs/3.0.0/sql-ref-functions-builtin.html#window-functions)

### Create a DataFrame that Interpolates the Broken Values

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, lag, lead

dateWindow = Window.orderBy("p_eventdate")

interpolatedDF = spark.read.table("health_tracker_plus_silver").select(
    "*",
    lag(col("heartrate")).over(dateWindow).alias("prev_amt"),
    lead(col("heartrate")).over(dateWindow).alias("next_amt"),
)

### Create a DataFrame of Updates

In [0]:
updatesDF = interpolatedDF.where(col("heartrate") < 0).select(
    "device_id",
    ((col("prev_amt") + col("next_amt")) / 2).alias("heartrate"),
    "eventtime",
    "name",
    "p_eventdate",
)

display(updatesDF)

device_id,heartrate,eventtime,name,p_eventdate
4,101.21254699240001,2020-01-01T18:59:12.000+0000,James Hou,2020-01-01
1,79.4468974614,2020-01-02T10:59:12.000+0000,Kristin Vasser,2020-01-02
1,92.10732860255,2020-01-04T20:01:04.000+0000,Kristin Vasser,2020-01-04
1,54.18606982715,2020-01-06T23:00:16.000+0000,Kristin Vasser,2020-01-06
1,78.55314577125,2020-01-07T22:00:32.000+0000,Kristin Vasser,2020-01-07
2,86.8898895492,2020-01-09T15:00:16.000+0000,Sam Knopp,2020-01-09
4,104.77145415449999,2020-01-09T03:58:56.000+0000,James Hou,2020-01-09
2,97.2776487597,2020-01-12T18:59:12.000+0000,Sam Knopp,2020-01-12
4,63.68804357805,2020-01-12T01:59:28.000+0000,James Hou,2020-01-12
4,105.83667578194999,2020-01-12T08:00:00.000+0000,James Hou,2020-01-12


**Exercise:** Write an assertion to verify that the Silver table and the UpdatesDF have the same schema

In [0]:
# TODO
assert spark.read.table("health_tracker_plus_silver").schema == updatesDF.schema, "Schemas do not match"
print("Assertion passed.")

Assertion passed.


## Update the Silver Table

In [0]:
from delta.tables import DeltaTable

silverTable = DeltaTable.forPath(spark, silverPath)

update_match = """
  health_tracker.eventtime = updates.eventtime
  AND
  health_tracker.device_id = updates.device_id
"""

update = {"heartrate": "updates.heartrate"}

(
    silverTable.alias("health_tracker")
    .merge(updatesDF.alias("updates"), update_match)
    .whenMatchedUpdate(set=update)
    .execute()
)

## Handle Late-Arriving Data

🤦🏼‍It turns out that our expectation of receiving the missing records late was correct. The complete month of February has subsequently been made available to us.

In [0]:
retrieve_data(2020, 2, rawPath, is_late=True)

Out[32]: True

In [0]:
display(dbutils.fs.ls(rawPath + "/late"))

path,name,size
dbfs:/dbacademy/gaurav_chattree/dataengineering/plus/raw/late/health_tracker_data_2020_2_late.json,health_tracker_data_2020_2_late.json,290497


**Exercise:** Count the records in the late file

The late file is a json file in the `rawPath + "late"` directory.

In [0]:

spark.read.json(rawPath + "/late").count()

Out[34]: 3480

🧐 You should note that the late file has all the records from the month of February, a count of 3480.

❗️ If we simply append this file to the Bronze Delta table, it will create many duplicate entries.

## Read the Late File

Next we read in the late file. Note that we make use of the `transform_raw` function loaded from the `includes/main/python/operations` notebook.

In [0]:
kafka_schema = "value STRING"

lateRawDF = spark.read.format("text").schema(kafka_schema).load(rawPath + "/late")

transformedLateRawDF = transform_raw(lateRawDF)

## Merge the Late-Arriving Data with the Bronze Table

We use the special method `.whenNotMatchedInsertAll` to insert only the records that are not present in the Bronze table. This is a best practice for preventing duplicate entries in a Delta table.

In [0]:
bronzeTable = DeltaTable.forPath(spark, bronzePath)

existing_record_match = "bronze.value = latearrivals.value"

(
    bronzeTable.alias("bronze")
    .merge(transformedLateRawDF.alias("latearrivals"), existing_record_match)
    .whenNotMatchedInsertAll()
    .execute()
)

**Exercise:** Write An Aggregation on the Silver table

### Count the number of records in the Silver table for each device id

💪🏼 The Silver table is registered in the Metastore as `health_tracker_plus_silver`.

👀 **Hint**: We did this exact query in the previous notebook.

In [0]:
# TODO
from pyspark.sql.functions import count

display(
spark.read.table("health_tracker_plus_silver").groupby("device_id").agg(count("*"))
)

device_id,count(1)
1,1440
3,1440
4,1440
2,1440
0,1440


### Troubleshooting

😫 If you run this query before the stream from the Bronze to the Silver tables has been picked up you will still see missing records for `device_id`: 4.

Wait a moment and run the query again.

### Check Yourself

You should see that there are an equal number of entries, 1440, for each device id.

## Table Histories

In [0]:
display(bronzeTable.history())

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
2,2022-03-25T06:37:18.000+0000,8047228571528786,gchattre@ur.rochester.edu,MERGE,"Map(predicate -> (bronze.`value` = latearrivals.`value`), matchedPredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(3719025789001905),0325-052223-ncmvp8x,1,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, executionTimeMs -> 2302, numTargetRowsInserted -> 72, scanTimeMs -> 0, numTargetRowsUpdated -> 0, numOutputRows -> 72, numTargetChangeFilesAdded -> 0, numSourceRows -> 3480, numTargetFilesRemoved -> 0, rewriteTimeMs -> 2226)",null
1,2022-03-25T06:02:37.000+0000,8047228571528786,gchattre@ur.rochester.edu,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 733fdb5a-68da-4b0c-9198-0ed8b0802f77, epochId -> 1)",null,List(3719025789001831),0325-052223-ncmvp8x,0,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 3408, numOutputBytes -> 71505, numAddedFiles -> 1)",null
0,2022-03-25T05:44:17.000+0000,8047228571528786,gchattre@ur.rochester.edu,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 733fdb5a-68da-4b0c-9198-0ed8b0802f77, epochId -> 0)",null,List(3719025789001790),0325-052223-ncmvp8x,null,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 3720, numOutputBytes -> 77601, numAddedFiles -> 1)",null


In [0]:
display(silverTable.history())

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
2,2022-03-25T06:37:23.000+0000,8047228571528786,gchattre@ur.rochester.edu,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 9949bb4c-54af-4293-ba39-319b3e5e0f8d, epochId -> 1)",null,List(3719025789001905),0325-052223-ncmvp8x,1,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 72, numOutputBytes -> 5116, numAddedFiles -> 3)",null
1,2022-03-25T06:34:00.000+0000,8047228571528786,gchattre@ur.rochester.edu,MERGE,"Map(predicate -> ((health_tracker.`eventtime` = updates.`eventtime`) AND (health_tracker.`device_id` = updates.`device_id`)), matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [])",null,List(3719025789001905),0325-052223-ncmvp8x,0,WriteSerializable,false,"Map(numTargetRowsCopied -> 4692, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 40, executionTimeMs -> 31184, numTargetRowsInserted -> 0, scanTimeMs -> 13804, numTargetRowsUpdated -> 60, numOutputRows -> 4752, numTargetChangeFilesAdded -> 0, numSourceRows -> 60, numTargetFilesRemoved -> 40, rewriteTimeMs -> 17275)",null
0,2022-03-25T06:05:18.000+0000,8047228571528786,gchattre@ur.rochester.edu,STREAMING UPDATE,"Map(outputMode -> Append, queryId -> 9949bb4c-54af-4293-ba39-319b3e5e0f8d, epochId -> 0)",null,List(3719025789001831),0325-052223-ncmvp8x,null,WriteSerializable,true,"Map(numRemovedFiles -> 0, numOutputRows -> 7128, numOutputBytes -> 152784, numAddedFiles -> 60)",null


## Time Travel
We can query an earlier version of the Delta table using the time travel feature. By running the following two cells, we can see that the current table count is larger than it was before we ingested the new data file into the stream.

In [0]:
%sql

SELECT COUNT(*) FROM health_tracker_plus_silver VERSION AS OF 0

count(1)
7128


In [0]:
%sql

SELECT COUNT(*) FROM health_tracker_plus_silver VERSION AS OF 1

count(1)
7128


In [0]:
%sql

SELECT COUNT(*) FROM health_tracker_plus_silver VERSION AS OF 2

count(1)
7200


## Stop All Streams

In the next notebook, we will build the Silver to Gold Step.

Before we do so, let's shut down all streams in this notebook.

In [0]:
stop_all_streams()


Out[41]: True

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>